In [1]:
!pip install opencv-python

In [2]:
!pip install keras

In [3]:
!pip install deepface

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
from deepface import DeepFace
import random

In [ ]:
from flask import Flask,render_template,request,url_for,redirect,Response,session
import cv2
from deepface import DeepFace

global aadharno,capture
capture=0
myapp = Flask(__name__)
cam = cv2.VideoCapture(0)
@myapp.route('/')
def homepage():
    return render_template("homepage.html")

@myapp.route('/',methods=['POST'])
def store():
    global aadharno,name,phone
    aadharno = request.form.get('Aadhar')
    name = request.form.get('Uname')
    phone = request.form.get('phone')
    session["aadharno"] = aadharno
    session["name"] = name
    session["phone"] = phone
    return redirect(url_for('camera'))
    
   
@myapp.route('/camera')
def camera():
    return render_template('camera.html')


def gen():
    global capture
    while True:
        suc,frame = cam.read()
        if suc:
            if(capture):
                capture=0
                p='img.jpg'
                cv2.imwrite(p,frame)
            else:     
                try:
                    ret, buffer = cv2.imencode('.jpg', frame)
                    frame = buffer.tobytes()
                    yield (b'--frame\r\n'
                       b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')    
                except Exception as e:
                    pass
            
        else:
            break
            
@myapp.route('/requests',methods=['POST'])
def tasks():
    if request.method == 'POST':
        if request.form.get('capture') == 'capture':
            global capture
            capture=1
            global aadharno,name,phone
            if "aadharno" in session:
                aadharno = session["aadharno"]
                if "name" in session:
                    name = session["name"]
                if "phone" in session:
                    phone = session["phone"]
                #print(aadharno)
                try:
                    aadhar_image = cv2.imread('static/data/' + aadharno + '.jpg')
                    p = cv2.imread('img.jpg')
                    result = DeepFace.verify(p,aadhar_image,model_name='Facenet',enforce_detection=False,
                         distance_metric='euclidean_l2',detector_backend = 'retinaface')
                    print(result)
                    if result['verified']:
                        return render_template('verified.html',n=name,p=phone,a=aadharno)
                    else:
                        return redirect('/failure')
                except:
                    return redirect('/failure')
                
    return render_template('camera.html')

@myapp.route('/video')
def video():
    return Response(gen(),mimetype='multipart/x-mixed-replace; boundary=frame')

@myapp.route('/success')
def success():
    return render_template('verified.html',n=name,p=phone)

@myapp.route('/failure')
def failure():
    return render_template('notverified.html')



if __name__=="__main__":
    myapp.secret_key= 'super secret key'
    myapp.config['SESSION_TYPE']='filesystem'
    myapp.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [ ]:
import ipywidgets as widgets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
from deepface import DeepFace
import random
import cv2 as cv
def check():
    video = cv.VideoCapture(0)
    aadhar_no = input("Enter Aadhar Number:")
    
    while True:
        ret, image = video.read()
    #c = plt.imread("sample2.jpg")

        aadhar_image = 'static/data/' + aadhar_no + '.jpg'

        result = DeepFace.verify(image,aadhar_image,model_name="Facenet",enforce_detection=False,
                         distance_metric='euclidean_l2',detector_backend = 'retinaface')
     
        print(result['verified'])
    #cv.imshow("Video",cv.cvtColor(image,cv.COLOR_BGR2RGB))
    
    
    
        if cv.waitKey(1) and 0xFF == ord('q'):
            break
        
    video.release()
    cv.destroyAllWindows()
    
l = [random.randint(1,100) for i in range(10)]
k = list(set(l))
## Here i is time in minutes
for i in range(1,100):
    if i in k:
        check()
        print(i,"Checked")